# Instrumentation and Control: Intro to Computer Vision 
## Wednesday November 28th, 2018
### Created by Amanda Boatswain Jacques (code modified from PyimageSearch Gurus) 

## Introduction 

We'll write a simple super fast object tracking that can run in real time, easily obtaining 32+ FPS on modern hardware systems.
We are going to focus on how to detect and track objects of different color in an image. We will try to track a green ball and and blue colored ball. 

### Import Library Packages 

Among the various packages we will be using OpenCV, which is a free computer vision and image processing package, and imutils, a package of helper functions for handling images. 

##### (Fun fact: The imutils package was developed by the creator of this tutorial!)

In [2]:
# import the necessary packages
import imutils
import os 
import cv2

##  Color Segmentation 

We are going to perform a process called color-segmentation. In short, we are going to manipulate an image in a way where pixels of a given color will be turned on, while others will be turned off. We will be using the HSV (Hue-Saturation-Value) color space. 

In [3]:
# define the color ranges for green and blue 
colorRanges = [ 
    ((29, 86, 6), (64, 255, 255), "green"),
    ((57, 68, 0), (151, 255, 255), "blue")]


## Grab Image Data 

We will search through a folder of images with the balls in them and try to locate and track them. We can see the list of images by printing them below: 

In [4]:
directory = "./object_tracking/ball_pictures/"

## Image Preprocessing 

Next we are going to read each image in the folder and apply some image processing techniques to find the balls. 
These include: 
* Conversion to the HSV color space
* Blur the image a bit to even out the pixels (noise removal)
* Erosion and dilation which help to remove small blobs in the mask 

We can see what each of these operations do by using a sample image. 

In [ ]:
imgpath =  directory + "/ball_2452.jpg"
image = cv2.imread(imgpath)

# blur the frame and convert it to the HSV color space
blurred = cv2.GaussianBlur(image, (11, 11), 0)
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

cv2.imshow("Original Image", image)
cv2.imshow("Blurred", blurred)
cv2.imshow("HSV", hsv)

# wait until window is closed 
cv2.waitKey(10000) 
cv2.destroyAllWindows()

## Thresholding 

Now let's see the results of the color segmentation and erosion/dilation. 

In [ ]:
imgpath =  directory + "/ball_2452.jpg"
image = cv2.imread(imgpath)

# blur the frame and convert it to the HSV color space
blurred = cv2.GaussianBlur(image, (11, 11), 0)
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

# loop over the color ranges
for (lower, upper, colorName) in colorRanges:
    # construct a mask for all colors in the current HSV range, then
    # perform a series of dilations and erosions to remove any small
    # blobs left in the mask
    mask = cv2.inRange(hsv, lower, upper)
    
    # we can adjust the number of dilations to see the difference 
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    print("Currently evaluating %s mask." %colorName)
    cv2.imshow("Original Image", image)
    cv2.imshow("Masked", mask)
    
    # wait until window is closed 
    cv2.waitKey(5000) 
cv2.destroyAllWindows()

Looks good!  Now we can do the same thing with the entire dataset. 

## Final Results 

We are then going to find the contours of the balls using some handy functions in OpenCV and then track them using a circle that we will draw directly on the image. 

In [5]:
# loop through all the images in the working directory 
for root, dirs, filenames in os.walk(directory):
    for i, file in enumerate(filenames):
        imgpath = os.path.join(root,file) # Reconstructs the file path using
        # the root_directory and current filename
        
        frame = cv2.imread(imgpath)
        # resize the frame, blur it, and convert it to the HSV color space
        frame = imutils.resize(frame, width=600)
        blurred = cv2.GaussianBlur(frame, (11, 11), 0)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # loop over the color ranges
        for (lower, upper, colorName) in colorRanges:
            # construct a mask for all colors in the current HSV range, then
            # perform a series of dilations and erosions to remove any small
            # blobs left in the mask
            mask = cv2.inRange(hsv, lower, upper)
            mask = cv2.erode(mask, None, iterations=2)
            mask = cv2.dilate(mask, None, iterations=2)
            
            # find contours in the mask
            cnts, hierarchy = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)
            #cnts = cnts[0] if imutils.is_cv2() else cnts[1]
            
            # only proceed if at least one contour was found
            if len(cnts) > 0:
                # find the largest contour in the mask, then use it to compute
                # the minimum enclosing circle and centroid
                c = max(cnts, key = cv2.contourArea)
                ((x, y), radius) = cv2.minEnclosingCircle(c)
                M = cv2.moments(c)
                (cX, cY) = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                
                # only draw the enclosing circle and text if the radious meets
                # a minimum size
                if radius > 10:
                    cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                    cv2.putText(frame, colorName, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
                                1.0, (0, 255, 255), 2)
        # show the frame to our screen
        cv2.imshow("Frame", frame)
        
        # wait half a second between frames 
        cv2.waitKey(500) 
        
cv2.destroyAllWindows()